# Predicting Reddit R-rated Content by Title
The goal of this analysis is to be able to predict by the title if the post is going to be intended for people over the age of 18 or it will be suitable for anyone.

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer



In [2]:
#Read Dataset that queried from SQL 
df1 = pd.read_csv("Over18Content.csv")
df2 = pd.read_csv("Under18Content.csv")

In [3]:
#Concatenate two datasets
frames = [df1, df2]

df3 = pd.concat(frames)

In [4]:
#Number of rows and columns
df3.shape

(2000, 2)

# Tokenize and Clean Data

In [5]:
Titles = []

for row in df3["title"]:
    #tokenize words
    words = word_tokenize(row)
    #remove punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    #remove stop words
    english_stops = set(stopwords.words('english'))
    clean_words = [word for word in clean_words if word not in english_stops]
    #Lematise words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    Titles.append(lemma_list)
    


In [6]:
#Convert each column to list and zip together to prepare it for bag of words

In [7]:
Age_Allowed = []
for row in df3["over_18"]:
    Age_Allowed.append(row)

In [8]:
print(len(Titles))
print(len(Age_Allowed))

2000
2000


In [9]:
#Sample row
print(Titles[1500])
print(Age_Allowed[1500])

['ending', 'place', 'never', 'thought', 'would']
False


In [38]:
#Combine both lists
combined = zip(Titles,Age_Allowed)

In [11]:
#Define bag_of_words function
def bag_of_words(words):
    return dict([(word, True) for word in words])

In [12]:
#Classify each word into True (Over 18) or False (Under 18)
Final_Data = []
for r, v in combined:
    bag_of_words(r)
    Final_Data.append((bag_of_words(r),v))
    
#Sample of what dataset looks like    
print(Final_Data[1000:1003]) 

[({'call': True, '866': True, '746-5265': True, 'hottest': True, 'flight': True, 'deals-this': True, 'special': True, 'reddit': True, 'deal': True, 'site': True, 'trip': True, 'cancun': True, '550': True, 'miami': True, '250': True, 'cost': True, 'rica': True, '459': True, 'la': True, 'vega': True, '375': True, 'mention': True, 'get': True, '10': True, '1st': True, 'ticket': True, 'purchase': True, 'limited': True, 'seating': True, 'reserve': True, 'www': True}, False), ({'hiring': True, 'individual': True, 'active': True, 'top': True, 'secret': True, 'sci': True, 'clearance': True, 'expired': True, 'within': True, 'past': True, '12': True, 'month': True, 'vandenberg': True, 'air': True, 'force': True, 'base': True, 'california': True, '15.50': True, '30/hr': True, 'benefit': True}, False), ({'hiring': True, 'individual': True, 'active': True, 'top': True, 'secret': True, 'ts/sci': True, 'clearance': True, 'expired': True, 'past': True, '12': True, 'month': True, 'vandenberg': True, 'a

In [22]:
#Shuffle records 
import random
random.shuffle(Final_Data)
print(len(Final_Data))

2000


## Build Naive Bayes Model

In [23]:
# Split the dataset into training and test subsets
train_set, test_set = Final_Data[0:1400], Final_Data[1400:]

import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics



refsets3 = collections. defaultdict(set)
testsets3 = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)

 
for i, (feats, label) in enumerate(test_set):
    refsets3[label].add(i)
    observed = classifier.classify(feats)
    testsets3[observed].add(i)


print("Naive Bayes Performance with Unigrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))




classifier.show_most_informative_features(n=10)

Naive Bayes Performance with Unigrams 
Accuracy: 0.8716666666666667
Most Informative Features
                      lf = True             True : False  =     17.9 : 1.0
                     fun = True             True : False  =     16.5 : 1.0
                 request = True             True : False  =     15.8 : 1.0
                    chat = True             True : False  =     15.8 : 1.0
                    girl = True             True : False  =     13.8 : 1.0
               subreddit = True             True : False  =     11.1 : 1.0
                      19 = True             True : False  =      9.8 : 1.0
                      24 = True             True : False  =      9.1 : 1.0
                question = True            False : True   =      8.0 : 1.0
                      18 = True             True : False  =      7.8 : 1.0


## Null Accuracy to Compare to Model

In [24]:
Counter = 0
False_Counter = 0
for i, v in test_set:
    if v == True:
        Counter += 1 
    else:
        False_Counter += 1 
        
#Create counter to count how many records in test set were true vs how many were false
print("True:", Counter)
print("False:",False_Counter)

print("Null Accuracy:", (Counter / (Counter + False_Counter)) * 100)

True: 304
False: 296
Null Accuracy: 50.66666666666667


## Result

The null accuracy for the test set is 50.43%. The naive bayes model accuractely classified 84.17% of the records, which is  33.74 % better than a naive prediction without any classifiers.

Analyzing most informative features:
The term "lf" appears 18.8 times in a over_18 post than under_18. Urban dictionary says "lf" means "Looking for".

Words such as chat, fun, subreddit, and request also have a higher chance of appearing in over_18 posts.

The numbers 22 and 24 most likely represent the age of a person, which you can see an example below. 

The words "campaign", "idea", and "v" are most likely to signal under_18 post. 

"v" stands for video upload.


In [16]:
twenty_four_example = df3[df3['title'].str.contains("24")]
print(twenty_four_example.head(5))

                                                 title  over_18
27   24[M4F] Online/Anywhere - Bi dude searching fo...     True
114  Vendor review - Imperialstormtrooper's GG249 2...     True
184  24 [M4MF] Long Island/NYC Looking to watch a c...     True
225      24 [F4m] Domme seeking bi male slave under 35     True
426  Any sissies want a dom black top to control th...     True


In [17]:
#Fun is a word that kids use a lot, but adults use it a lot in a different way.
#This word shows the importance of making sure to mark content correctly. 
#You don't want a young child opening post by mistake from an adult looking for "fun"

fun_example = df3[df3['title'].str.contains("fun")]
print(fun_example.head(4))
print(fun_example.tail(4))

                                                 title  over_18
25   30 [M4F] Portland or Online - Seeking flirty, ...     True
73         21 [M4F] German boy looking for some fun :)     True
252                      [B/S] Pool time fun: Update 9     True
263                     [B/S] Pool time fun: Update 10     True
                                                 title  over_18
726  Which F.Priest is better overall? (dps, end ga...    False
899  What are some of the most fun magical items/ar...    False
924  5e: looking for suggestions for lvl 1 items th...    False
946  What are some fun/Wacky things you have done a...    False


# Naive Bayes using Bigrams
### Build a model with bigrams to see how it will perform comapred to unigrams

In [39]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.metrics import TrigramAssocMeasures
from nltk import bigrams, trigrams
import nltk
from nltk.collocations import *

def bag_of_bigrams_words(words, score_fn=BigramAssocMeasures.chi_sq, n = 200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return bag_of_words(bigrams)

bigram_all_data = [(bag_of_bigrams_words(i), label) for (i, label) in combined]



In [62]:
import random
random.shuffle(bigram_all_data)
print(len(bigram_all_data))

2000


In [63]:
# Split the dataset into training and test subsets
train_set, test_set = bigram_all_data[:1800], bigram_all_data[1800:]


import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics
from nltk.classify import DecisionTreeClassifier


refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)


 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

#precision - Out of the ones our model said will be (pos/neg), how many did we predict correctly ? 
#recall - Out of the ones that were (pos/neg), how many did we predict correctly? 
print("Naive Bayes Performance with Ngrams Combined ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))


classifier.show_most_informative_features(n=9)

Naive Bayes Performance with Ngrams Combined 
Accuracy: 0.575
Most Informative Features
         ('xbox', 'one') = True            False : True   =      4.4 : 1.0
        ('want', 'play') = True             True : False  =      3.6 : 1.0
      ('please', 'help') = True             True : False  =      3.0 : 1.0
           ('let', "'s") = True             True : False  =      3.0 : 1.0
        ('need', 'help') = True            False : True   =      2.5 : 1.0
         ('year', 'old') = True            False : True   =      2.3 : 1.0
   ('quick', 'question') = True            False : True   =      2.3 : 1.0
       ('would', 'like') = True             True : False  =      2.3 : 1.0
      ('looking', 'get') = True             True : False  =      2.3 : 1.0


# Bigrams Results

A bigrams only model performs with 57.5 % accuracy, which is 26.67% lower than the unigrams only model. Nonetheless, it is still interesting to see the key words that appear in over 18 posts compared to suitable for everyone.